In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = '/content/dataset/dataset'  # Tüm verilerin bulunduğu ana klasör
batch_size = 32
img_width, img_height = 128, 128  # Görüntü boyutları

# Data augmentation ve validation split için ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # %20 doğrulama, %80 eğitim için ayrılacak
)

# Eğitim verisi için generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Eğitim için subset
)

# Doğrulama verisi için generator
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Doğrulama için subset
)

# Sınıfların sayısını kontrol et
print("Sınıf Sayısı:", train_generator.num_classes)
print("Sınıf Etiketleri:", train_generator.class_indices)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, TimeDistributed, Flatten, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

img_width, img_height = 128, 128  # Görüntü boyutları
lstm_units = 64  # LSTM hücre sayısı
dropout_rate = 0.5  # Dropout oranı
learning_rate = 0.001  # Öğrenme oranı

# CNN + LSTM Modeli
model = Sequential()

# Removing TimeDistributed and adjusting input_shape
# input_shape is now (img_width, img_height, 3) for individual images
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

# If you still need LSTM for some temporal features,
# you might need to reshape the output of Flatten to
# have a time dimension before feeding it to LSTM.
# This would require further modifications to your data and model architecture.

# Çıkış katmanı
model.add(Dense(128, activation='relu'))
model.add(Dense(train_generator.num_classes, activation='softmax'))  # Sınıf sayısını burada alıyoruz

# Modeli derleme
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli özetle
model.summary()

In [ ]:
# Early stopping callback eklenebilir
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Doğrulama setinde tahminler yapma
validation_generator.reset()
predictions = model.predict(validation_generator, steps=validation_generator.samples // batch_size + 1)
y_pred = predictions.argmax(axis=1)

# Gerçek etiketleri alma
y_true = validation_generator.classes

# Performans metriklerini hesaplama
print('Classification Report:')
print(classification_report(y_true, y_pred, target_names=validation_generator.class_indices.keys()))

# Confusion matrix (Karışıklık matrisi) ile inceleme
print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred))


In [ ]:
import numpy as np
# Test veya doğrulama verileri üzerinde tahmin yapma
# 'validation_generator' kullanarak tahminleri alıyoruz.
y_true = validation_generator.classes  # Gerçek etiketler
y_pred_prob = model.predict(validation_generator)  # Tahmin edilen olasılıklar

# En yüksek olasılığa göre sınıfları belirle
y_pred = np.argmax(y_pred_prob, axis=1)

# Metrikleri hesaplayın
precision = precision_score(y_true, y_pred, average='weighted')  # 'macro' veya 'micro' da kullanabilirsiniz
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
accuracy = accuracy_score(y_true, y_pred)

# Sonuçları yazdırın
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")